### 1.Transformer为何使用多头注意力机制?

使用多头注意力机制可以从多空间获得信息，如果模型只使用单一的注意力头，它可能只会捕捉序列中一些特定的模式，但使用多头注意力机制可以更好地平衡模型对不同模式的关注程度。

### 2.Transformer为什么Q和K使用不同的权重矩阵生成，为何不能使用同一个值进行自身的点乘?

因为 Q 和 K 的职责和作用不同，它们的表示需要满足不同的条件。K和Q使用了不同的W_k, W_Q来计算，可以理解为是在**不同空间上的投影**。正因为有了这种不同空间的投影，增加了表达能力，这样计算得到的attention score矩阵的泛化能力更高。例如，Q 的表示需要包含单词本身的语义信息，而 K 的表示需要包含单词与其他单词之间的相关性信息。如果使用同一个权重矩阵来生成 Q 和 K，那么它们的表示可能会缺乏其中一个方面的信息，导致模型的性能不够理想。
在进行点乘时也需要使用不同的值，这样才能使模型的表示能够满足不同的需求，同时可以增强表达能力，提高泛化能力。

### 3.Transformer计算attention的时候为何选择点乘而不是加法? 两者计算复杂度和效果上有什么区别?

为了计算更快，在表达相似度上加法没有点积表现好，而且attention计算就是点积运算，但是随着维度d的增大，加性模型会明显好于点积模型

### 4.为什么在进行softmax之前需要对attention进行scaled (为什么除以dk的平方根)，并使用公式推导进行讲解

假设q和k的各个分量是相互独立的随机变量，它们的均值为0，方差为1，那么qk经过点积之后的均值依然为0，但是方差变为了dk，所以要除以根号dk，把它拉回到μ=0，σ=1的分布中来。
公式：https://zhuanlan.zhihu.com/p/149634836

![](python_work/Attachments_pics/2023-03-12-16-17-10.png)

$$ \text{Softmax}(z_i) = \frac{\exp(z_i)}{\sum_j \exp(z_j)} $$

以数组为例，2个长度是len，均值是0，方差是1的数组点积会生成长度是len，均值是0，方差是len的数组。而**方差变大会导致softmax的输入推向正无穷或负无穷**，这时的**梯度会无限趋近于0，不利于训练的收敛**。因此除以len的开方，可以是数组的方差重新回归到1，有利于训练的收敛。

### 5.在计算attention score的时候如何对padding做mask操作?

**padding_mask**：分头后，在计算注意力得分时，生成一个**encoding_padding_mask矩阵**，需要mask的位置为1，其他位置为0，然后整个矩阵乘以一个`-np.inf`值，然后和经过缩放的q@k矩阵相加（自动广播），然后通过softmax时padding部分的`-np.inf会变为0`，即不参与计算
![](python_work/Attachments_pics/2023-03-12-17-18-14.png)

**sequence_mask**是对角线为0，右上部分为1，然后整个矩阵乘以一个`-np.inf`值，然后和经过缩放的q@k矩阵相加（自动广播），然后通过softmax时padding部分的`-np.inf会变为0`，即不参与计算

mask是将一些不要用的值掩盖掉，使其不产生作用。有两种mask，第一种是**padding mask**，在所有scaled dot-product attention都用到；第二种是**sequence mask**，在decoder的self-attention里面用到。

**padding mask**：因为一个批量输入中，所有序列的长度是不同的。为了符合模型的输入方式，会用padding的方式来填充（比如填0），使所有序列的长度一致。但填充部分是没有意义的，所以在计算注意力的时候，不需要也不应该有注意力分配到这些填充的值上面。所以解决方式就是在填充的位置赋予一个很小的负值/负无穷（-np.inf）的值，经过softmax后的得分为0，即没有注意力分配到这个上面。

def padding_mask(seq_k, seq_q):  

    # shape(seq_k)=(B,L_k), shape(seq_q)=(B,L_q)  

    # 因为要计算seq_k和seq_q的相似程度，来表示注意力的得分  

    # padding mask要作用在QK^T上，所以padding mask是跟seq_k和seq_q序列长度相关的矩阵  

    # shape(padding mask)=(B, L_q, L_k)  

    len_q = seq_q.size(1)  

    # `PAD` is 0，这里要计算seq_k序列中，padding为0的地方，并将相应位置变为True，方便后面处理  

    pad_mask = seq_k.eq(0)  

    # 将每个seq_k序列扩展len_q次，shape=[B, L_q, L_k]  

    pad_mask = pad_mask.unsqueeze(1).expand(-1, len_q, -1)  
    
    return pad_mask
以上方法为大部分padding mask的计算形式，但实际上，这里做了seq_q全部有效的假设（没有padding），并不够精确 。自己的看法：上述代码expand操作，只是将seq_k中padding的部分重复了L_q次，并没有注意到，seq_q也有padding的部分。即在一个(L_q,L_k)矩阵中，只有最后几列需要掩码，实际矩阵的最后几行也需要掩码。（以后上图更形象）

**sequence mask**：在decoder部分，因为不能见到下文信息（防止泄漏），所以用mask的方式掩盖掉当前时刻t及之后的下文信息。具体，可产生一个对角线为0的上三角矩阵，将其作用到每个decoder的输入列上。代码如下：

def sequence_mask(seq):

    batch_size, seq_len = seq.size()

    mask = torch.triu(torch.ones((seq_len, seq_len), dtype=torch.uint8),diagonal=1)

    mask = mask.unsqueeze(0).expand(batch_size, -1, -1)  # [B, L, L]

    # 三角矩阵中，为1的部分是需要被掩码掉的
    
    return mask

* decoder-block有两个multi-head attention，下面的multi-head attention是目标输入的self-attention，需要用到1.padding mask：去除padding位置的影响；2.sequence mask：去掉下文穿越的影响。上面的multi-head attention只需要padding mask，因为下面的多头注意力已经磨平了下文信息。当encoder和decoder的输入序列长度一样时，可以通过padding mask+sequence mask作为scaled dot-product attention的attn_mask来实现。
* 其他情况的attn_mask（代码中的表达）等于padding mask

### 6.为什么在进行多头注意力的时候需要对每个head进行降维?

如果只使用 one head 并且维度为$d_{model}$，相较于 8 head 并且维度为$d_{model}$ / 8 。首先存在计算量极大的问题，并且高维空间下的学习难度也会相应提升，这就难免文中实验出现的参数量大且效果不佳的情况，于是将原有的高维空间转化为多个低维空间并再最后进行拼接，形成同样维度的输出，借此丰富特性信息，降低了计算量，而且取得了更好的效果，十分巧妙。

### 7.大概讲一下Transformer的Encoder模块。

它由多个encoder层组成，每个层包含多个头的多头注意力机制和一个前馈神经网络。
在进行编码时，encoder层会对输入序列进行处理，得到输入序列的内部表示，每个encoder层中的多头注意力机制会根据输入序列的位置信息，计算出不同位置的词的重要性。然后通过前馈神经网络，将这些重要信息与输入序列的词向量相结合，得到输入序列新的表示。
最终，整个模块的encoder层会被拼接，得到最终的输入序列的内部表示。这个内部表示可以用作模型的后续计算，如分类、生成等任务。

### 8.为何在获取输入词向量之后需要对矩阵乘以embeddding size的开方？意义是什么？

embedding matrix的初始化方式是**xavier init**，这种方式的方差是**1/embedding size**，因此乘以embedding size的开方使得embedding matrix的方差是1，在这个scale下可能更有利于embedding matrix的收敛。

### 9.简单介绍一下Transformer的位置编码? 有什么意义和优缺点?

![](python_work/Attachments_pics/2023-03-12-17-53-09.png)

**pos**：相当于token

**i**： 用**词向量的维度的index地板除（//）2取整得到的值**，[0，0，1，1，2，2...]

根据position的index处于是单数（运用sin）双数（运用cos）计算位置信息




在没有 Position embedding 的 Transformer 模型并不能捕捉序列的顺序，交换单词位置后 attention map 的对应位置数值也会进行交换，并不会产生数值变化，即没有词序信息。所以这时候想要将词序信息加入到模型中。
  
  $$ \text{PE}(\text{pos}, i) = \sin\left(\frac{\text{pos}}{10000^{\frac{2i}{\text{Dmodel}}}}\right) $$
  
  $$ \text{PE}(\text{pos}, i+1) = \cos\left(\frac{\text{pos}}{10000^{\frac{2i}{\text{Dmodel}}}}\right) $$
  
我们可以看到只要位置和词向量维度确定了,词的绝对位置和相对位置就能确定，因为transformer是并行化操作的，那么就会像cnn那样没有了顺序，此时添加位置编码，就可以知道上下文关系，但是它会在自注意机制经过线性变化后消失
具体实现方法：首先，为每一个词分配一个位置编码向量，该向量的维度和词向量的维度相同。然后将词向量乘以一个大数之后与位置编码向量相加得到词的最终向量表示。
优点：它能有效地考虑词语词与词之间的相对位置关系，提高模型的准确度。
缺点：计算过程中需要额外的计算，会增加模型的计算复杂度。

### 10.你还了解哪些关于位置编码的技术，各自的优缺点是什么?

固定位置编码：这种方法是将每个词的位置信息编码成一个固定的词向量，并与词向量相加得到最终词向量。优点是简单易实现，缺点是无法对位置信息进行训练和微调。

### 11.简单讲一下Transformer中的残差结构以及意义。

把输入和经过变化后的输出一起传递给下一个结构，它的意义很简单，在反向传播求导时，**保证已经有了一个固定项，即保证了梯度不会消失**.

### 12.为什么transformer块使用LayerNorm而不是BatchNorm? LayerNorm 在Transformer的位置是哪里?

**LN对同一样本的不同维度作归一化** 

**LayerNorm的计算公式**可以表示为：

$$ y = \frac{x - E(x)}{\sqrt{Var(x) + \epsilon}} * \gamma + \beta $$

其中，$E(x)$表示输入$x$的期望值，$Var(x)$表示输入$x$的方差，$\epsilon$是一个很小的数值（用于防止分母为0），$\gamma$和$\beta$是可学习的参数。

**BN不同样本的相同维度作归一化**

因为这是在进行nlp任务，同一个样本包含的是同一个词向量的信息，对它整体变化并不会改变它的信息，另外batch_size很小的情况下它并不能代表整体，它的位置在encoder，decoder里每一个输出层后面，对输出数据进行归一化操作，提高模型的训练效果。

### 13.简答讲一下BatchNorm技术，以及它的优缺点。

**BN优点**:
1. 使得损失更加平滑，加快收敛 

2. 缓解梯度饱和问题（针对sigmoid问题）  

**BN缺点**：batch_size小的时候，**用整个batch的均值和方差来模拟全部样本的均值和方差，其并不具有代表性**.

### 14.简单描述一下Transformer中的前馈神经网络? 使用了什么激活函数? 相关优缺点?

答：前馈神经网络中使用了多层感知机，主要是计算多头注意力的权重值，以及对输入数据进行非线性变换。
用了Gelu函数，它能够Relu函数非线性特性的同时，提高神经网络的训练效果

### 15. Encoder端和Decoder端是如何进行交互的?

encoder端和decoder端之间是通过中间层进行交互的。中间层会接收来自encoder端的语义表示，并将其传递给decoder端。decoder端会根据语义表示来生成输出语句。这样，encoder端和decoder端就可以相互协作，实现自然语言的翻译、摘要等任务。
通过encoder出来的值，只取k和v，然后和decoder里经过masked attention出来的q相反应，以翻译为例，就是进行两种语言的交互。

### 16. Decoder阶段的多头自注意力和encoder的多头自注意力有什么区别?

Encoder的多头注意力机层主要根据词的**上下文信息计算权重值**  

Decoder的多头注意力机层则会根据词的**语义信息和上下文信息来计算权重值**

#### 17.Transformer的并行化提现在哪个地方? Decoder端可以做并行化吗?

**并行在多头注意力机制中**，这种设计的优点是：可以并行计算提升计算效率，此外，由于每个头都是独立的，所以可以根据实际需求调整头的数量，以满足不同的需求。

### 18.简单描述一下wordpiece model 和 byte pair encoding，有实际应用过吗?

wordpiece是一种分词技术，它将一个词分为多个子词，以更好地表示词的语义信息。例如将"book" 拆分为 "book" 和 "##ing"，表示这个词是一个动词而不是名词。
byte pair encoding是将重复的字符压缩为较短的符号，来减少字符串的长度，例如，将字符串 "hello world" 压缩为 "h@@ w@@"，表示 "@@" 代表字符 "ello"。

### 19. Transformer训练的时候学习率是如何设定的? Dropout是如何设定的，位置在哪里? Dropout在测试的需要有什么需要注意的吗?

在训练过程中，会根据模型的表现来**动态调整学习率**，当模型的表现较好时，会增加学习率，以**加快模型的收敛速度**；当模型表现不佳时，会降低学习率，以**避免陷入局部最优解**。这种动态调整学习率的方法可以帮助transformer模型更快地收敛，同时避免了固定学习率带来的可能的问题，如收敛速度过慢或陷入局部最优解。  

**Dropout**是一种常见的正则化方法，它在训练时随机让一部分神经元失活，以**防止过拟合**，在训练transformer时，通常在每一层的**输入和输出上使用dropout**。例如，在训练自注意力层时，你可能会在每一层的输入和输出上使用dropout，以保护模型的输入和输出不会过拟合。

**在测试时，通常不使用dropout**，这是因为dropout是用来防止过拟合的，而测试数据不应该包含过拟合模型。因此，在测试时应该要禁用dropout，以便模型能够充分利用它所有的神经元，并且对测试数据进行准确的预测。

### 20.引申一个关于bert问题，bert的mask为何不学习transformer在attention处进行屏蔽score的技巧?

BERT和transformer的目标不一致，bert是语言的预训练模型，需要充分考虑上下文的关系，而transformer主要考虑句子中第i个元素与前i-1个元素的关系。

### 21.不考虑多头的原因，self-attention中词向量不乘QKV参数矩阵，会有什么问题？

self_attention的核心是利用文本中的其他词来增强目标词的语义表示，从而更好地利用上下文信息，如果不乘q，模型无法得到语义信息，不乘k，模型无法的词与词之间的关系，不乘v，模型无法得到词的上下文信息，总得来说不乘QKV参数矩阵会导致模型失去理解语言的能力。

### 22.为什么BERT选择mask掉15%这个比例的词，可以是其他的比例吗？

类似完型填空的做法，也和CBOW有异曲同工之妙。如果mask的词过少，模型就会学习到一些不太有用的信息，如果mask的词过多，模型无法从上下文中学习到足够的信息，15%是一个合理的选择，但是，无论选择什么比例，都要保证模型能够从上下文中学习到足够的信息。

### 23.使用BERT预训练模型为什么最多只能输入512个词，最多只能两个句子合成一句？

是为了显存考虑，即计算资源。

### 24.为什么BERT在第一句前会加一个[CLS]标志?

**BERT在第一句前会加一个[CLS]标志，最后一层该位对应向量可以作为整句话的语义表示，从而用于下游的分类任务等。**

为什么选它呢，因为与文本中已有的其它词相比，这个无明显语义信息的符号会更“公平”地融合文本中各个词的语义信息，从而更好的表示整句话的语义。

具体来说，self-attention是用文本中的其它词来增强目标词的语义表示，但是目标词本身的语义还是会占主要部分的，因此，经过BERT的12层，每次词的embedding融合了所有词的信息，可以去更好的表示自己的语义。

**而[CLS]位本身没有语义，经过12层，得到的是attention后所有词的加权平均，相比其他正常词，可以更好的表征句子语义。**

当然，也可以通过对最后一层所有词的embedding做pooling去表征句子语义。

这里补充一下bert的输出，有两种，在BERT TF源码中对应：

一种是get_pooled_out()，就是上述[CLS]的表示，输出shape是[batch size,hidden size]。

一种是get_sequence_out()，获取的是整个句子每一个token的向量表示，输出shape是[batch_size, seq_length, hidden_size]，这里也包括[CLS]，因此在做token级别的任务时要注意它。

### 25.Transformer在哪里做了权重共享，为什么可以做权重共享？

（1）Encoder和Decoder间的Embedding层权重共享；

（2）Decoder中Embedding层和FC层权重共享。

### 26.BERT非线性的来源在哪里？

Self_attention和GeLU是非线性的。

### 27.BERT 是如何区分一词多义的？

 BERT 使用一种称为“**上下文无关嵌入**”的技术，通过考虑一个单词在句子中的上下文来区分该单词的不同用法。 例如，在句子“我喜欢吃香蕉”中，单词“香蕉”指的是一种水果，而在句子“这是一个香蕉形状的东西”中，单词“香蕉”指的是一个具有香蕉形状的东西。 BERT 通过考虑单词的上下文，可以区分它们不同的意义。

### 28. BERT中，为什么已经有85%不被mask的单词，还需要在15%进行“mask策略”的单词中选取10%保持原词不变？

在**mask策略**选中的15%中：

* **80%的token被替换为 [MASK]**

**意义**：在不泄露label的情况下，通过被mask的单词的上下文，预测该单词，训练模型和词向量。

**问题**：下游任务中不会出现 [MASK]，导致预训练和微调不一致。



* **10% 的 token会被替换为随机的 token**

**意义**：`减弱了预训练和微调不一致给模型带来的影响`。同时让模型知道，当单词不是[MASK]标记时，仍然需要输出，就迫使模型尽量在每一个token（即使该词不是mask）上都学习基于上下文的表示，这也是**解决一词多义的关键**。

**问题**：模型可能会认为要预测的词永远不会是该位置原本的词。



* **10% 的 tokens 会保持不变但需要被预测**

**意义**：让模型知道要预测的词有可能是该位置原本的词，而不是永远都是随机单词。



### 29.Bert的基本原理

bert整体是一个自编码语言模型，采用两个任务来训练模型：

1.MaskLM  2.NSP

缺点：
1. 模型参数过多，少量数据训练时容易过拟合 

2. 预训练时会采用mask策略，而downstream任务没有，就会产生一种不匹配

3. bert分词时会把word分成几部分，probability 这个词被切分成”pro”、”#babi”和”#lity”3 个 WordPiece。有可能出现的一种随机 Mask 是把”#babi” Mask 住，但是”pro”和”#lity”没有被 Mask。（可以用Whole Word Masking解决）

### 30.Bert是怎么用Transformer的

$Bert_{base}$采用12层transformer的encoder结构共110M个参数，$Bert_{large}$采用了24层的encoder结构共340M个参数，它比transformer的encoder多了一个token_type_ids(针对NSP任务的)

### 31.BERT 应用于有空格丢失或者单词拼写错误等数据是否还是有效？有什么改进的方法？

空格丢失就相当于是一段长文本，不确定它是否能处理，但是为了保险起见我们可以先作Bi-LSTM+CRF分词处理，再输入bert作下游任务。

### 32.BERT 的 embedding 向量如何得来的？

对于输入的 token embedding、segment embedding、position embedding 都是随机生成的，需要注意的是在 **Transformer 论文中的 position embedding 由 sin/cos 函数生成的固定的值，而bert中是跟普通 word embedding 一样随机生成的，可以训练的**。作者这里这样选择的原因可能是 BERT 训练的数据比 Transformer 那篇大很多，完全可以让模型自己去学习。

### 33. 为什么 BERT 中输入数据的[mask]标记为什么不能直接留空或者直接输入原始数据，在 self-attention 的 Q K V 计算中，不与待预测的单词做 Q K V 交互计算？

1. 如果输入原始数据，则整个输入序列都将参与到 Q K V 的计算中，这样会让模型**过于关注待预测单词周围的上下文信息**，而导致模型在处理长文本时出现**计算效率低下**的问题。

2. 如果直接留空，则模型**无法感知到预测单词的存在**，无法进行预测，而且会使得**整个序列的信息丢失**。

3. 在 BERT 中，使用 [MASK] 标记来替换一部分单词可以使得模型在预测时仅仅关注待预测单词周围的上下文信息，而不需要关注整个序列。同时，为了防止模型过于依赖 [MASK] 标记周围的信息，BERT 还引入了另外一种训练任务，即下一句预测（Next Sentence Prediction, NSP），通过让模型预测两个句子之间的关系，来训练模型对整个句子的理解。这样，BERT 就可以更好地处理长文本，并且可以学习到更好的上下文表示。

### 34.BERT 的两个预训练任务对应的损失函数是什么(用公式形式展示)？

BERT的损失函数就是**交叉熵损失函数**，BERT 的损失函数由两部分组成，第一部分是来自 Mask-LM 的「单词级别分类任务」，另一部分是「句子级别的分类任务」。通过这两个任务的联合学习，可以使得 BERT 学习到的表征**既有 token 级别信息，同时也包含了句子级别的语义信息**。具体损失函数如下：

L(θ,$θ_1$,$θ_2$) = $L_1$(θ,$θ_1$) + $L_2$(θ,$θ_2$)

具体的预训练工程实现细节方面，BERT 还利用了一系列策略，使得模型更易于训练，比如对于**学习率的 warm-up 策略**，使用的**激活函数**不再是普通的 ReLu，而是 **GeLu**，也使用了**dropout** 等常见的训练技巧。

### 35.BERT 的三个 Embedding 直接相加会对语义有影响吗？

Embedding的本质是以one hot为输入的单层全连接层。现在将token，position，segment三者都用one hot 表示。然后concat起来，然后通过单层全连接，等价效果就是三个Embedding相加，BERT 的三个 Embedding 相加，本质可以看作一个**特征的融合，可以学到融合后特征的语义信息**

### 36.什么是词袋模型（Bag of Words Model）？有什么优缺点？
词袋模型是一种在信息检索中使用的模型。它假定dui'yu一个文本，忽略其词序和语法、句法，将其仅仅看成是一个词集合，或者词的组合。**文本中的每一个词都是独立的，不依赖于其他词是否出现。**

**优点**：基于频率统计方法，**易于理解**

**缺点***：向量维度高，导致后续相似度或文本分类的**计算非常大**，并且**稀疏数据也会导致相似度区分不明显**。

### 37.请介绍一下 Word2Vec 模型的原理和作用。

Word2Vec 是一种语言模型，它是从大量文本预料中以**无监督**方式学习语义知识的模型，被广泛地应用于自然语言处理中。它的作用就是**将自然语言中的字词转为计算机可以理解的稠密向量**（Dense Vector）。

Word2Vec 的原理就是**一个词预测前后词或者前后词预测当前词，使得概率最大化**。它的训练模型本质上是只具有一个隐含层的神经元网络。它的输入是采用 One-Hot 编码的词汇表向量，它的输出也是 One-Hot 编码的词汇表向量。使用所有样本训练这个神经元网络，等到收敛之后，**从输入层到隐含层的那些权重就可以作为每个单词对应的稠密向量**。

### 38.如何处理一个文本分类任务？可以说一下具体的流程吗？
1. 首先，需要对文本数据进行预处理，包括分词、去除停用词、标点符号等。

2. 然后，需要将文本数据转换为计算机可以理解的数值形式。这可以通过使用词袋模型（Bag of Words Model）、TF-IDF 等方法来实现。

3. 接下来，可以选择一个合适的分类算法，例如朴素贝叶斯、支持向量机（SVM）、决策树等。

4. 在训练模型之前，可以使用交叉验证等方法来选择最优的模型参数。

最后，在训练好的模型上对新的文本数据进行分类。

DisTilBERT解决一切！

### 39.请问如何评估一个情感分析模型的性能？有哪些指标可以使用？

1. 准确率（accuracy）：表示模型预测正确的样本数占总样本的比例

2. 召回率（recall）：在正样本中，模型预测为正样本的比例

3. 精确率（precession）：模型预测为正样本，实际为正样本的比例

4. F1 值：是精确度和召回率的调和平均数，用于综合衡量模型的性能
![](python_work/Attachments_pics/2023-03-12-19-48-07.png)

### 40.在 NLP 中，你如何处理词义歧义和同义词问题？可以举一个例子说明吗？

对于同义词问题，NLP通常使用**向量空间模型**来表示单词的语义信息。这种方法可以将具有相似含义的单词映射到向量空间中相近的位置。

假设我们有一个句子：“我喜欢吃苹果。”在这个句子中，“苹果”一词可能指水果，也可能指苹果公司的产品。为了确定“苹果”一词的正确含义，NLP系统会根据上下文信息进行判断。由于句子中出现了“吃”这个动词，所以系统会判断“苹果”一词指代水果。

### 41.什么是词嵌入（Word Embedding）？有哪些常见的词嵌入模型？

词嵌入是一种将词汇**映射到连续向量空间**的技术。它在深度学习中是为了下游任务（序列标记或文本分类）提供输入特征。
包括Word2Vec、GloVe、FastText等。这些模型都可以用来学习词汇的向量表示。

GloVe（Global Vectors for Word Representation）是一种无监督的词嵌入方法，它的目标是进行词的向量化表示，使得向量之间尽可能多地蕴含语义和语法的信息。

GloVe模型首先基于语料库构建词的共现矩阵，然后基于**共现矩阵**和 GloVe 模型学习词向量。它结合了全局**矩阵分解和局部滑动窗口**方法的优点。

### 42. 在 NLP 中，你如何处理命名实体识别（NER）任务？可以说一下具体的方法吗？
有多种方法可以用来处理 NER 任务。其中一些常见的方法包括基于规则的方法、基于统计的方法和基于深度学习的方法。例如，可以使用隐马尔可夫模型（HMM）、条件随机场（CRF）等机器学习算法来进行序列标注。

BERT+（LSTM）+CRF

### 43.你在实际的 NLP 项目中遇到过哪些挑战？你是如何克服这些挑战的？
transformer优化

### 44. NLP 中有哪些方法可以做数据增强
1. 可以使用**文本生成**技术来生成新的文本数据；

2. 可以使用**回译技术**将文本翻译成另一种语言，然后再翻译回来；

3. 也可以使用数据扩充技术，如**同义词替换**、随机插入、随机交换和随机删除等。

### 45.在文本分类任务中，如何对不平衡的数据集进行处理？
在文本分类任务中，如果数据集存在不平衡性，即某些类别的样本数量明显少于其他类别，这会对模型的训练和评估带来很大的影响。为了处理不平衡的数据集，可以采用以下一些方法：

1. 重新采样：包括欠采样和过采样。**欠采样**是指**随机删除**一些多数类别的样本以平衡数据集，而**过采样**则是通过**复制一些少数类别**的样本来增加其数量。这两种方法都有一定的缺陷，欠采样可能会丢失一些重要信息，而过采样可能会导致模型过拟合。因此，通常会采用一些高级的重新采样方法，如**SMOTE**等。

2. 类别权重：可以**为每个类别分配一个权重，使得损失函数在训练时更加重视少数类别的样本**。这种方法可以在不改变数据集的情况下平衡数据集。

3. 阈值调整：在测试时，可以根据实际需求调整分类阈值，使得分类器更加关注少数类别的样本。

4. 集成学习：可以将多个不同的模型结合起来进行预测，以减少对于少数类别样本的误分类。

### 46.请简单介绍一下文本生成任务中的语言模型和条件语言模型。
**语言模型**是一种用于**预测文本序列中下一个词的概率分布**的模型。它可以用于文本生成任务，通过不断预测下一个词来生成新的文本。

条件语言模型则是在**给定某些条件**（如话题、情感等）的情况下，预测文本序列中下一个词的概率分布。它可以用于生成特定话题或情感的文本。

总之，语言模型和条件语言模型都可以用于文本生成任务，但条件语言模型可以生成**更符合特定条件**的文本。

### 47. 在 NLP 中，你如何处理情感分析任务？可以举一个例子说明吗？
我直接一个roberta-base-uncased，finetune一个model

### 48. 你觉得最近 NLP 领域出现了哪些重要的技术和研究方向？为什么这些技术和方向很重要？


目前，NLP 领域的一些重要研究方向包括自然语言理解、人机对话和自然语言生成。